In [1]:
import sklearn
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Data loading

In [3]:
data_raw = pd.read_csv('compas/raw/compas-scores-two-years.csv')

In [4]:
columns_for_training = ["sex", "race", "juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count", "c_charge_degree", "decile_score", "age_cat"]

responder = data_raw["is_recid"]
data = data_raw[columns_for_training]

In [5]:
data.isna().sum()

sex                0
race               0
juv_fel_count      0
juv_misd_count     0
juv_other_count    0
priors_count       0
c_charge_degree    0
decile_score       0
age_cat            0
dtype: int64

In [6]:
numerical_columns = ['juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'decile_score', 'age_cat']
categorical_columns = ["sex", "race", "c_charge_degree"]
categorical_one_hot = pd.get_dummies(data[categorical_columns])
numerical = data[numerical_columns]
X = pd.concat([numerical, categorical_one_hot], axis=1)

In [7]:
X_train_extended, X_test_extended, y_train, y_test = train_test_split(X, responder, test_size=0.2, random_state=42)
X_train = X_train_extended.drop(columns=["age_cat"])
X_test = X_test_extended.drop(columns=["age_cat"])

# Fairness metrics calculation methods

In [8]:
def get_statistical_parity(y_true, y_pred, sensitive_features):
    positive_rate_protected = y_pred[sensitive_features == 1].mean()
    positive_rate_privileged = y_pred[sensitive_features == 0].mean()
    return positive_rate_privileged / positive_rate_protected

In [9]:
def get_equal_opportunity(y_true, y_pred, sensitive_features):
    def get_tpr(_y_true, _y_pred):
        return np.sum((_y_true == 1) & (_y_pred == 1)) / np.sum(_y_true == 1)
    
    protected_pred, protected_true = y_pred[sensitive_features == 1], y_true[sensitive_features == 1]
    privileged_pred, privileged_true = y_pred[sensitive_features == 0], y_true[sensitive_features == 0]
    return get_tpr(privileged_true, privileged_pred) / get_tpr(protected_true, protected_pred) 

In [10]:
def get_predictive_rate_parity(y_true, y_pred, sensitive_features):
    protected_pos_parity = y_true[(sensitive_features == 1) & (y_pred == 1)].mean()
    protected_neg_parity = y_true[(sensitive_features == 1) & (y_pred == 0)].mean()
    privileged_pos_parity = y_true[(sensitive_features == 0) & (y_pred == 1)].mean()
    privileged_neg_parity = y_true[(sensitive_features == 0) & (y_pred == 0)].mean()
    return privileged_pos_parity / protected_pos_parity, privileged_neg_parity / protected_neg_parity

# Random forest

In [11]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

rf_pred = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("Random Forest Classification Report:\n", classification_report(y_test, rf_pred))

Random Forest Accuracy: 0.6257796257796258
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.66      0.66       789
           1       0.59      0.58      0.59       654

    accuracy                           0.63      1443
   macro avg       0.62      0.62      0.62      1443
weighted avg       0.63      0.63      0.63      1443



# XGBoost

In [12]:
xgb_model = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

xgb_pred = xgb_model.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_pred))
print("XGBoost Classification Report:\n", classification_report(y_test, xgb_pred))

XGBoost Accuracy: 0.6541926541926542
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.69      0.68       789
           1       0.62      0.61      0.62       654

    accuracy                           0.65      1443
   macro avg       0.65      0.65      0.65      1443
weighted avg       0.65      0.65      0.65      1443



# Logistic Regression

In [13]:
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)

lr_pred = lr_model.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_pred))
print("Logistic Regression Classification Report:\n", classification_report(y_test, lr_pred))

Logistic Regression Accuracy: 0.6666666666666666
Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.72      0.70       789
           1       0.64      0.60      0.62       654

    accuracy                           0.67      1443
   macro avg       0.66      0.66      0.66      1443
weighted avg       0.67      0.67      0.67      1443



# Fairness evaluation

In [14]:
X_test_extended["age_cat"].unique()

array(['Greater than 45', '25 - 45', 'Less than 25'], dtype=object)

In [15]:
protected = 'Greater than 45'
sensitive_features = X_test_extended["age_cat"] == protected

def evaluate_model(prediction, true_val, sensitive_feature):
    pos, neg = get_predictive_rate_parity(true_val, prediction, sensitive_feature)
    print("Statistical Parity:", get_statistical_parity(true_val, prediction, sensitive_feature) * 100)
    print("Equal Opportunity:", get_equal_opportunity(true_val, prediction, sensitive_feature) * 100)
    print(f"Predictive Rate Parity: positive {pos*100}, negative {neg*100}")
    
print("Random Forest:")
evaluate_model(rf_pred, y_test, sensitive_features)
print("XGBoost:")
evaluate_model(xgb_pred, y_test, sensitive_features)
print("Logistic Regression:")
evaluate_model(lr_pred, y_test, sensitive_features)

Random Forest:
Statistical Parity: 178.0154486036839
Equal Opportunity: 138.6609780111585
Predictive Rate Parity: positive 123.45779220779221, negative 158.5409252669039
XGBoost:
Statistical Parity: 177.69756387403447
Equal Opportunity: 130.03757459662563
Predictive Rate Parity: positive 115.9870030301924, negative 161.70724062271447
Logistic Regression:
Statistical Parity: 184.5669753488979
Equal Opportunity: 129.36221419975936
Predictive Rate Parity: positive 111.09010712035288, negative 162.76430146581916


In [16]:
y_train[X_train_extended["age_cat"] == protected].mean(), y_train[X_train_extended["age_cat"] != protected].mean()

(0.349003984063745, 0.5267936226749336)

In [17]:
desired_ratio = y_train[X_train_extended["age_cat"] != protected].mean()
current_ratio = y_train[X_train_extended["age_cat"] == protected].mean()
true_positives_protected =  y_train[(X_train_extended["age_cat"] == protected) & (y_train == 1)]
to_upsample = int(((desired_ratio/current_ratio) - 1) / (1 - desired_ratio) * len(true_positives_protected))
aditional_samples = X_train_extended[(X_train_extended["age_cat"] == protected) & (y_train == 1)].sample(n=to_upsample, replace=True)
augmented_X_train_extended = pd.concat([X_train_extended, aditional_samples]).reset_index(drop=True)
augmented_X_train = augmented_X_train_extended.drop(columns=["age_cat"])
augmented_y_train = pd.concat([y_train, pd.Series([1] * to_upsample)]).reset_index(drop=True)

In [18]:
augmented_y_train[augmented_X_train_extended["age_cat"] == protected].mean(), augmented_y_train[augmented_X_train_extended["age_cat"] != protected].mean()

(0.5266512166859791, 0.5267936226749336)

In [19]:
# Initialize and train the Random Forest classifier# Initialize and train the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(augmented_X_train, augmented_y_train)

# Predict and evaluate
rf_pred_fair = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred_fair))
print("Random Forest Classification Report:\n", classification_report(y_test, rf_pred_fair))


Random Forest Accuracy: 0.6174636174636174
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.62      0.64       789
           1       0.57      0.61      0.59       654

    accuracy                           0.62      1443
   macro avg       0.62      0.62      0.62      1443
weighted avg       0.62      0.62      0.62      1443



In [20]:
print("Random Forest:")
evaluate_model(rf_pred_fair, y_test, sensitive_features)

Random Forest:
Statistical Parity: 156.95187165775403
Equal Opportunity: 131.61853188929
Predictive Rate Parity: positive 132.9145371947757, negative 156.92307692307693


In [21]:
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(augmented_X_train, augmented_y_train)

lr_pred_fair = lr_model.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_pred_fair))
print("Logistic Regression Classification Report:\n", classification_report(y_test, lr_pred_fair))

Logistic Regression Accuracy: 0.665973665973666
Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.67      0.69       789
           1       0.62      0.66      0.64       654

    accuracy                           0.67      1443
   macro avg       0.66      0.67      0.66      1443
weighted avg       0.67      0.67      0.67      1443



In [22]:
print("Logistic regression:")
evaluate_model(lr_pred_fair, y_test, sensitive_features)

Logistic regression:
Statistical Parity: 157.62284330541823
Equal Opportunity: 120.87416193914386
Predictive Rate Parity: positive 121.5447651663405, negative 162.57603920243326


In [23]:
xgb_model = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(augmented_X_train, augmented_y_train)

xgb_pred_fair = xgb_model.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_pred_fair))
print("XGBoost Classification Report:\n", classification_report(y_test, xgb_pred_fair))

XGBoost Accuracy: 0.647955647955648
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.67      0.67       789
           1       0.61      0.63      0.62       654

    accuracy                           0.65      1443
   macro avg       0.65      0.65      0.65      1443
weighted avg       0.65      0.65      0.65      1443



In [24]:
print("XGBoost:")
evaluate_model(xgb_pred_fair, y_test, sensitive_features)

Logistic regression:
Statistical Parity: 159.877319911922
Equal Opportunity: 135.75511432009628
Predictive Rate Parity: positive 134.58333333333331, negative 147.25125418060202
